In [5]:
import pandas as pd
import numpy as np
dataset=pd.read_csv("ckd.csv")


In [6]:
dataset=pd.get_dummies(dataset, drop_first=True)
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [7]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]



In [8]:

dependent=dataset['classification_yes']


In [9]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [14]:
from sklearn.preprocessing import StandardScaler
SC=StandardScaler()
x_train=SC.fit_transform(x_train)
x_test=SC.transform(x_test)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid= {'n_estimators':[10,100,1000],'criterion':['gini', 'entropy', 'log_loss'], 'min_samples_split':[2,20,100],'min_samples_leaf':[0.1,1.0,10.0,100.0],'max_features':['sqrt','log2'],'class_weight':['balanced','balanced_subsample']}

grid=GridSearchCV(RandomForestClassifier(),param_grid,scoring='f1_weighted')
grid.fit(x_train,y_train)

C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
1800 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "C:\Users\poorn\Anaconda3\lib\site-packages\joblib\parallel.py", line 921, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\poorn\Anaconda3\lib\s

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'class_weight': ['balanced', 'balanced_subsample'],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [0.1, 1.0, 10.0, 100.0],
                         'min_samples_split': [2, 20, 100],
                         'n_estimators': [10, 100, 1000]},
             scoring='f1_weighted')

In [16]:
# print best parameter after tuning 
#print(grid.best_params_) 


In [17]:
re=grid.cv_results_
print(re)


{'mean_fit_time': array([0.00725074, 0.04617038, 0.42740402, 0.00532942, 0.04328518,
       0.42276516, 0.00547328, 0.04013958, 0.39673915, 0.00257816,
       0.02071109, 0.19882774, 0.00313931, 0.02076254, 0.19845614,
       0.00283446, 0.02084799, 0.19918294, 0.00280461, 0.02041764,
       0.19887509, 0.00299153, 0.02104516, 0.19940515, 0.00275559,
       0.02108097, 0.19985843, 0.00287709, 0.02184095, 0.20345607,
       0.00317183, 0.02082448, 0.2037293 , 0.00266304, 0.02164793,
       0.21050057, 0.00528779, 0.04469023, 0.41909428, 0.00547667,
       0.05042391, 0.43070784, 0.00520349, 0.0422543 , 0.41425567,
       0.00283442, 0.02071385, 0.20415425, 0.00321288, 0.02127995,
       0.2137392 , 0.00286932, 0.02143664, 0.20826907, 0.00287328,
       0.02152534, 0.20940552, 0.00310793, 0.02191586, 0.20780058,
       0.00302677, 0.02192507, 0.21064949, 0.00288606, 0.02169027,
       0.20909753, 0.00270495, 0.02243891, 0.21005111, 0.00306106,
       0.02197528, 0.21082048, 0.00536046, 0

In [18]:
grid_predictions = grid.predict(x_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
cm


array([[50,  1],
       [ 6, 76]], dtype=int64)

In [19]:

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print(clf_report)

              precision    recall  f1-score   support

           0       0.89      0.98      0.93        51
           1       0.99      0.93      0.96        82

    accuracy                           0.95       133
   macro avg       0.94      0.95      0.95       133
weighted avg       0.95      0.95      0.95       133



In [20]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'class_weight': 'balanced', 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 2, 'n_estimators': 100}: 0.9477705902916346


In [21]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])


0.9966523194643712

In [22]:
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)
print((cm))
print(clf_report)

The f1_macro value for best parameter {'class_weight': 'balanced', 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 2, 'n_estimators': 100}: 0.9477705902916346
[[50  1]
 [ 6 76]]
              precision    recall  f1-score   support

           0       0.89      0.98      0.93        51
           1       0.99      0.93      0.96        82

    accuracy                           0.95       133
   macro avg       0.94      0.95      0.95       133
weighted avg       0.95      0.95      0.95       133



In [23]:
table=pd.DataFrame.from_dict(re)

In [24]:
table.shape

(432, 19)

In [25]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007251,0.003166,0.001458,0.000433,balanced,gini,sqrt,0.1,2,10,"{'class_weight': 'balanced', 'criterion': 'gin...",0.944707,0.923510,0.944023,0.981031,0.961826,0.951019,0.019303,45
1,0.046170,0.003776,0.005227,0.000143,balanced,gini,sqrt,0.1,2,100,"{'class_weight': 'balanced', 'criterion': 'gin...",0.963284,0.962264,0.907035,0.981031,1.000000,0.962723,0.031069,1
2,0.427404,0.007580,0.045711,0.001103,balanced,gini,sqrt,0.1,2,1000,"{'class_weight': 'balanced', 'criterion': 'gin...",0.926567,0.943041,0.907035,0.981031,1.000000,0.951535,0.034336,35
3,0.005329,0.000368,0.001142,0.000207,balanced,gini,sqrt,0.1,20,10,"{'class_weight': 'balanced', 'criterion': 'gin...",0.962963,0.923510,0.869925,0.962264,0.981031,0.939939,0.039721,64
4,0.043285,0.000315,0.005427,0.000999,balanced,gini,sqrt,0.1,20,100,"{'class_weight': 'balanced', 'criterion': 'gin...",0.944707,0.924528,0.888515,0.981031,1.000000,0.947756,0.039737,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,0.021985,0.000710,0.000000,0.000000,balanced_subsample,log_loss,log2,100,20,100,"{'class_weight': 'balanced_subsample', 'criter...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,167
428,0.215091,0.000852,0.000000,0.000000,balanced_subsample,log_loss,log2,100,20,1000,"{'class_weight': 'balanced_subsample', 'criter...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,166
429,0.002990,0.000644,0.000000,0.000000,balanced_subsample,log_loss,log2,100,100,10,"{'class_weight': 'balanced_subsample', 'criter...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,165
430,0.022482,0.000689,0.000000,0.000000,balanced_subsample,log_loss,log2,100,100,100,"{'class_weight': 'balanced_subsample', 'criter...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,372
